In [38]:
#Matthew Lund
#mtlund@wpi.edu
#ECE577 Homework3

#imported libs

#For grabbing ImageNet Images
import os
script_directory = os.path.dirname(os.path.realpath('LundMatthew_ECE577_Proj3'))

#Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input

#Numpy and Plotting
import numpy as np
import matplotlib.pyplot as plt
#Using Jupyter Notebook
%matplotlib inline

#Foolbox
import foolbox
from foolbox.attacks import BlendedUniformNoiseAttack, FGSM, ContrastReductionAttack, SinglePixelAttack, SaliencyMapAttack

In [39]:
model = ResNet50(weights='imagenet')
preprocessing = (np.array([103.939, 116.779, 123.68]), 1)

# Foolbox criterion
attack_criterion = foolbox.criteria.Misclassification()

# Foolbox model
fmodel = foolbox.models.KerasModel(model, bounds=(0, 255), 
                                   preprocessing=preprocessing)

# Load 10 random images from ImageNet
image_paths = [
    '/Images/1.JPEG',
    '/Images/2.JPEG',
    '/Images/3.JPEG',
    '/Images/4.JPEG',
    '/Images/5.JPEG',
    '/Images/6.JPEG',
    '/Images/7.JPEG',
    '/Images/8.JPEG',
    '/Images/9.JPEG',
    '/Images/10.JPEG',
]

# Initialize attacks
attacks = {
    'BlendedUniformNoiseAttack': BlendedUniformNoiseAttack(fmodel),
    'ContrastReductionAttack': ContrastReductionAttack(fmodel),
    'FGSM': FGSM(fmodel),
    'SinglePixelAttack': SinglePixelAttack(fmodel),
    'SaliencyMapAttack': SaliencyMapAttack(fmodel)
}

In [40]:
# For Loop for Images
for image_file in image_paths:
    # Load image
    img_path = os.path.join(script_directory, image_file)
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    true_label = np.argmax(fmodel.predictions(img))
    # Apply each attack
    for attack_name, attack in attacks.items():
        target_label = (true_label + 1) % 1000
        adversarial = attack(fmodel, img, label=target_label)
        noise = adversarial - img

        # Plot original, adversarial, and noise
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title('Original')
        plt.imshow(img / 255.0)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title('Adversarial ({})'.format(attack_name))
        plt.imshow(adversarial / 255.0)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Noise')
        plt.imshow(noise / np.abs(noise).max() / 2.0 + 0.5)
        plt.axis('off')

        plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Images/1.JPEG'